### Friday, December 8, 2023

[TheBloke/Orca-2-13B-GPTQ](https://huggingface.co/TheBloke/Orca-2-13B-GPTQ)

This all runs. Nice!


In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
!ls /home/rob/Data2/huggingface/transformers

models--Intel--neural-chat-7b-v3-1
models--TheBloke--CodeLlama-34B-Instruct-GPTQ
models--TheBloke--Llama-2-13b-Chat-GPTQ
models--TheBloke--Orca-2-13B-GPTQ
models--TheBloke--Python-Code-33B-GPTQ
models--facebook--blenderbot-1B-distill
models--google--flan-t5-large
models--gpt2-medium
models--meta-llama--Llama-2-13b-chat-hf
models--meta-llama--Llama-2-13b-hf
models--model_name
tmp0m77pc0x
tmpcjh0h7gn
tmpfcrmwgx2
tmpzafytbf_
version.txt


In [4]:
# backup "TheBloke/Orca-2-13B-GPTQ"
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--Orca-2-13B-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 7.26GB to /home/rob/Data3/huggingface/transformers

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [5]:
model_name_or_path = "TheBloke/Orca-2-13B-GPTQ"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [7]:
%%time
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")
# load time 8610 MiB VRAM
# CPU times: user 8.73 s, sys: 2.75 s, total: 11.5 s
# Wall time: 4.75 s

# download time ...
# CPU times: user 36.6 s, sys: 38.8 s, total: 1min 15s
# Wall time: 1h 40min 27s

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


CPU times: user 12.8 s, sys: 3.4 s, total: 16.2 s
Wall time: 6.06 s


In [8]:
# This system message was copied from ... https://huggingface.co/microsoft/Orca-2-13b
system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."

In [9]:
prompt = "Tell me about AI"

prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''

In [10]:
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

In [11]:
%%time
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# 9892 MiB VRAM
# CPU times: user 16.2 s, sys: 207 ms, total: 16.4 s
# Wall time: 16.5 s


<s><|im_start|>system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.<|im_end|>
<|im_start|>user
Tell me about AI<|im_end|>
<|im_start|>assistant
AI stands for artificial intelligence, which is the branch of computer science that deals with creating machines or systems that can perform tasks that normally require human intelligence, such as reasoning, learning, decision making, perception, natural language processing, etc. AI can be divided into two main types: narrow AI and general AI. Narrow AI is the type of AI that can only perform a specific task or a subset of tasks, such as facial recognition, chess playing, speech recognition, etc. General AI is the type of AI that can perform any task that a human can, such as understanding natural language, common sense reasoning, creativity, etc. General AI does not exist yet,

In [12]:
# Inference can also be done using transformers' pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

In [13]:
print(pipe(prompt_template)[0]['generated_text'])

<|im_start|>system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.<|im_end|>
<|im_start|>user
Tell me about AI<|im_end|>
<|im_start|>assistant
AI stands for artificial intelligence, which is the field of computer science that studies how to create machines or systems that can perform tasks that normally require human intelligence, such as reasoning, learning, planning, problem-solving, perception, natural language processing, etc. AI can be classified into different types, such as narrow AI, which focuses on specific domains or applications, and general AI, which aims to achieve a universal intelligence that can adapt to any situation. Some examples of AI applications are chatbots, face recognition, self-driving cars, virtual assistants, games, recommendation systems, etc. AI also raises ethical, social, and philosophica